# FastText - Subword aware word2vec

This file is dedicated to experiments with FastText models. 

In [41]:
import pandas as pd 
import numpy as np 
from gensim.utils import simple_preprocess

import importlib 
import sys 
sys.path.append("../")

from proj_mod import fasttext 
importlib.reload(fasttext);

In [42]:
df_pt=pd.read_csv("../data/raw.csv")
df_pt

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN


In [43]:
FastTextRanker=fasttext.FastTextRanker()

In [44]:
FastTextRanker.fit(df=df_pt)

tfidf fitted. 
fasttest fitted. 


In [45]:
query=[
    "Aspiring human resources",
    "seeking human resources"
]
FastTextRanker.create_score(query=query)

Query embedding created. 
Corpus embedding created. 


In [47]:
FastTextRanker.df_fitted_.sort_values(by="new_score", ascending=False).head(10)

,id,job_title,old_score,new_score,fit_score
2,3,Aspiring Human Resources Professional,0,0.857855,0.857855
32,33,Aspiring Human Resources Professional,0,0.857855,0.857855
16,17,Aspiring Human Resources Professional,0,0.857855,0.857855
20,21,Aspiring Human Resources Professional,0,0.857855,0.857855
57,58,Aspiring Human Resources Professional,0,0.857855,0.857855
96,97,Aspiring Human Resources Professional,0,0.857855,0.857855
45,46,Aspiring Human Resources Professional,0,0.857855,0.857855
27,28,Seeking Human Resources Opportunities,0,0.824198,0.824198
29,30,Seeking Human Resources Opportunities,0,0.824198,0.824198
23,24,Aspiring Human Resources Specialist,0,0.812201,0.812201


In [48]:
query2=[
    "Seeking Human Resources Opportunities", 
    "Aspiring Human Resources Specialist"
]
FastTextRanker.create_score(query=query2)

Query embedding created. 
Using corpus embedding from the past. 


In [49]:
FastTextRanker.df_fitted_.sort_values(by="new_score", ascending=False).head(10)

,id,job_title,old_score,new_score,fit_score
27,28,Seeking Human Resources Opportunities,0.824198,0.850443,0.842569
29,30,Seeking Human Resources Opportunities,0.824198,0.850443,0.842569
5,6,Aspiring Human Resources Specialist,0.812201,0.850443,0.838970
23,24,Aspiring Human Resources Specialist,0.812201,0.850443,0.838970
35,36,Aspiring Human Resources Specialist,0.812201,0.850443,0.838970
48,49,Aspiring Human Resources Specialist,0.812201,0.850443,0.838970
59,60,Aspiring Human Resources Specialist,0.812201,0.850443,0.838970
2,3,Aspiring Human Resources Professional,0.857855,0.795164,0.813971
20,21,Aspiring Human Resources Professional,0.857855,0.795164,0.813971
57,58,Aspiring Human Resources Professional,0.857855,0.795164,0.813971


## Theory review of FastText 

The fundamentals of the FastText model is the same as the word2vec model, what differs is how FastText model handles "subwords". 
FastText embeds a word $w$ to an average of the vectors of its subwords. We will illustrate how this subword awareness is achieved with an example. 

Consider two words "actor" and "acting". 
* The *3-gram*'s of "actor" are: "<ac", "act", "cto", "tor", and "or>". 
* The 3-gram's of "acting" are: "<ac", "act", "cti", "tin", "ing", and "ng>". 

Notice how "<ac", and "act" appeared in both, this is how subwords are "made present". In this context, the *whole word token* is, for instance, "<acting>" for the word "acting". 

Now, give an arbitrary word $w$, we define the *set of wanted n-grams for $w$* $\mathcal{G}(w)$ as the set containing all wanted n-grams of $w$, together with the whole word token. In practice, we often set a range for n for the n-grams, for instance, we may set the range to $[3,6]$, then we will only consider 3,4,5,6-grams. 
For an n-gram $g$ of $w$, let $z_g$ be its vector. Then the embedded vector for word $w$ is 
$$
v(w):=\frac{1}{|\mathcal{G}(w)|}\sum\limits_{g\in\mathcal{G}(w)}z_g\; . 
$$

In this way, intuitively, we can achieve: 
* Related words share many n-grams, they should get embedded closer. 
* Unseen words might still have n-grams learned by the model, we can still build a vector for it. 


The rest is using these subword aware embeddings in place of the original word vectors in SGNS or CBOW-NS, I will not repeat the formats of them here, one can find them in the notebook "word2vec.ipynb". 

**Remark**: In SGNS, it is common to only apply the subword aware embedding to the center word vector. 